# **Overview**
This notebook documents the process of collecting user reviews from the Tokopedia Android application using the `google-play-scraper` Python library. The resulting dataset will be used for downstream tasks such as text cleaning, sentiment analysis, and topic modeling.

# **Objective**
To build a structured and comprehensive dataset of user-generated reviews from the Google Play Store, including ratings, review text, timestamps, and metadata.

In [1]:
!pip install --quiet google-play-scraper

# **Data Source**
- **Platform:** Google Play Store  
- **Library:** `google-play-scraper`  
- **App Package Name:** `com.tokopedia.tkpd`  
- **Data Retrieved:** Review text, score, review date, user metadata, and review IDs.

In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

# Core library
import pandas as pd

# Web scraping
from google_play_scraper import (
    Sort,
    reviews
)

pd.set_option('display.max_colwidth', None)

print('Ready!')

Ready!


# **Scraping Method**
We use the `reviews()` function from `google-play-scraper`, which provides a high-level interface for retrieving app reviews without the need to manually parse HTML or handle pagination.

# **Key Parameters**
- `lang`: Language of the reviews (`"id"` for Indonesian language review)
- `country`: Country store (`"id"` for Indonesian users)
- `sort`: Sorting method (`Sort.NEWEST` to get recent reviews)
- `count`: Number of reviews to fetch per request

# **Fetching Reviews in Batches**
The following cell fetches reviews in chunks of 100.000 using the continuation token returned by `reviews()`. All retrieved entries are stored in a list, which will later be converted into a DataFrame.

In [4]:
all_reviews = []

for _ in range(10):
    batch, continuation_token = reviews(
        'com.tokopedia.tkpd',
        lang='id',
        country='id',
        sort=Sort.NEWEST,
        count=100000,
        continuation_token=None
    )

    all_reviews.extend(batch)

    if continuation_token is None:
        break

    print(f'Successfully scraped {(_+1)*100000} data')

print('Data scraped successfully!')

Successfully scraped 20000 data
Successfully scraped 40000 data
Successfully scraped 60000 data
Successfully scraped 80000 data
Successfully scraped 100000 data
Successfully scraped 120000 data
Successfully scraped 140000 data
Successfully scraped 160000 data
Successfully scraped 180000 data
Successfully scraped 200000 data
Data scraped successfully!


In [5]:
df_scrape = pd.DataFrame(all_reviews)

# Choose only the necessary column
df_raw = df_scrape[['content', 'score', 'at']]
df_raw.columns = ['raw_text', 'rating', 'date']
df_raw.head(7)

,raw_text,rating,date
0,saat ini bagus,4,2025-11-29 00:30:03
1,mau beli barang pake verifikasi segala ribet,1,2025-11-29 00:10:21
2,bagus lagi,5,2025-11-28 23:46:25
3,"iklan di halaman pencarian produk sangat menggangu ketika mencari suatu produk ,selalu menampilkan iklan secara berulang ketika kita scroll produk",1,2025-11-28 23:40:18
4,oke bgt,5,2025-11-28 23:33:02
5,saya beli di Tokopedia pesanan 3 hari tidak di ambil oleh kurir dan dibalikan ke saldo refund gak langsung ke rekening ribet.. padahal mau ambil duid kita sendiri..,1,2025-11-28 22:23:59
6,mantap.....,5,2025-11-28 21:23:22


In [6]:
df_raw.to_csv('../data/raw/review.csv', index=False)
print('Scraped data successfully saved to "../data/raw/review.csv"')

Scraped data successfully saved to "../data/raw/review.csv"


# **Save Only Review File**

In [10]:
with open("../data/raw/all_reviews.txt", "w", encoding="utf-8") as f:
    for line in df_raw.raw_text.astype(str):
        f.write(line.replace("\n", " ") + "\n")

# **Limitations**
- Google Play may limit large-scale scraping; the library provides only accessible public data.
- Pagination relies on continuation tokens; if Google changes the API structure, functionality may break.
- Not guaranteed to retrieve **all** historical reviews due to API restrictions.
  
# **Next Steps**
- Clean and normalize the review text.
- Remove noise, emojis, and repeating characters.
- Perform exploratory analysis on ratings, review length, and temporal patterns.
- Apply sentiment classification, topic extraction, or clustering.